# Análisis de los datos provenientes del PDM

Este `jupyter` y los relacionados contiene el análisis y el proceso que permite la homologación/conversión entre los datos en formato `.csv` provenientes de la base de datos de los Planes de Desarrollo Municipal (PDM) de Honduras y el modelo de datos de Fonsagua.

En concreto en este `jupyter` se analizan los .csv proporcionados bajo distintos criterios, se procesan para adaptar la información de un modo más conveniente y se unen ambos `.csv` en un único _set_ de información para usos posteriores.

## Unión y Análisis de los datos de PDM

Los datos de PDM se proporcionan en dos ficheros `.csv`. El fichero `nasmar_dd.csv` contiene datos de población mientras que el fichero `nasmar.csv` contiene datos socioeconómicos.

A continuación se analiza si estos datos tienen un aspecto correcto en cuanto a:

* Columnas duplicadas
* Celdas en blanco
* Cabeceras
* Formato de los datos
* Incoherencias varias
* ...

Y se unen ambos csv para obtener un sólo sobre el que trabajar


In [1]:
# Configuración básica de jupyter, pandas, ...
# e importación de librerías que necesitaremos durante el análisis

# https://medium.com/@1522933668924/using-matplotlib-in-jupyter-notebooks-comparing-methods-and-some-tips-python-c38e85b40ba1
# Para hacer los mapas interactivos

# %matplotlib widget 
%matplotlib inline 
import numpy as np
from IPython.display import display, HTML, Markdown
# Ejemplos de uso
# display(HTML('<h2 style="color:red;">Subtitulo "YYY - NOMBRE"</h2>'))
# display(Markdown("# Título principal"))
# display(HTML(df.describe().to_html()))  # muestra todos los datos
import pandas
from ietl import pandas_utils

from ietl.geopandas_utils import (
    get_dataframe_from_spatialite_table,
    plot_polygon_labels,
)
import geopandas as gpd

pandas.options.mode.use_inf_as_na = True
# http://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
# pandas.set_option('display.max_rows', 500)
# pandas.set_option('display.max_columns', 500)


# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
# Si se está trabajando con módulos externos usar %autoreload 2 evita tener que
# recargar a mano. %autoreload 0 lo desactiva
%load_ext autoreload
%autoreload 2



# Ejecuta el .ipyndb referenciando como si fuera en este propio Lab
# Todas las variables del otro pasan a estar disponible en este
# Cualquier salida por pantalla del otro se muestra en los resultados 
# de la celda donde se ejecuta el %run
# if __name__ == '__main__' and '__file__' not in globals():
#     print("Este código sólo se ejecuta cuando lo llamo directamente y no a través de un `%run`")
# %run nasmar_cartografia.ipynb

In [ ]:
from nasmar import common_left_field, common_right_field
import nasmar

sqlite_path = "191208_fonsagua_todos_municipios.sqlite"
excel_datos_pdm_path = "pdm.xlsx"

In [2]:
# Leemos los datos de los ficheros csv y sanitizamos la cabecera
# left = nasmar.csv
# right = nasmar_dd.csv
left, right = nasmar.read_csv()

# Mostramos el número de filas y columnas
# Y comprobámos que el número de filas sea el mismo
pandas_utils.print_shape(left)
pandas_utils.print_shape(right)
pandas_utils.compare_n_rows(left, right)

# Comprobamos que dentro del mismo csv no hay columnas con el mismo nombre
pandas_utils.check_not_duplicated_columns_names(left)
pandas_utils.check_not_duplicated_columns_names(right)

# Comprobamos si hay celdas con valores en blanco
# print("Columnas con valores en blanco:", pandas_utils.get_na_column_names(left))
# print("Filas con valores en blanco:", pandas_utils.get_na_row_names(left))
print("Celdas con valores en blanco:\n", pandas_utils.get_positions_of_na(left))
# print("Columnas con valores en blanco:", pandas_utils.get_na_column_names(right))
# print("Filas con valores en blanco:", pandas_utils.get_na_row_names(right))
print("Celdas con valores en blanco:\n", pandas_utils.get_positions_of_na(right))

# Comprobamos que si hay columnas con el mismo nombre en ambos csv, tienen los
# mismos valores y obtenemos de nuevo los datos "sanitizados", sin columnas
# duplicadas iguales
sanitized_a, sanitized_b = pandas_utils.sanitize_remove_equal_columns(left, right, common_left_field, common_right_field)


# Unimos ambos csv
# https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
merge_options = {
    "left_on": common_left_field,
    "right_on": common_right_field,
    "how": "inner",
    "indicator": False,
    "validate": "one_to_one",
    "suffixes": (False, False),
}
merged = pandas.merge(sanitized_a, sanitized_b, **merge_options)
merged.name = "merged"

# "cod_municipio" no aporta nada. Dejamos el nombre para vistazos rápidos
merged.drop(columns="cod_municipio", inplace=True) 

# Comprobamos leading/trailing spaces que puedan estropear los datos
pandas_utils.check_spaces_from_dataframe(merged, raise_error=True)

# Comprobamos que no haya valores negativos. No tendría sentido para este caso
pandas_utils.check_negatives_from_dataframe(merged, raise_error=True)


# Mostramos el número de filas y columnas
# Y comprobámos que el número de filas sea el original 
pandas_utils.print_shape(merged)
pandas_utils.compare_n_rows(left, merged)

# Para echar un vistazo rápido a los tipos de de datos de las columnas
# print(sanitized_a.dtypes)
# print(sanitized_b.dtypes)
# print(merged.dtypes)

nasmar.csv. rows: 602, columns: 39
nasmar_dd.csv. rows: 602, columns: 14
Celdas con valores en blanco:
 [('60905015000', 'total_porcino'), ('60905015000', 'total_equino'), ('60609034000', 'total_ganosb'), ('60609034000', 'total_areatareas'), ('60609034000', 'total_riegotareas'), ('60609034000', 'total_bovino'), ('60609034000', 'total_equino'), ('60714034', 'total_bovino'), ('60714034', 'total_equino')]
Celdas con valores en blanco:
 []
merged. rows: 602, columns: 49


Las comprobaciones nos dicen que los datos al nivel básico un formato correcto. Únicamente se detectan algunas celdas en blanco pero no son significativas por lo que pueden obviarse

Listado de código de comunidad y columna que está en blanco:
    
* (60905015000, 'total_porcino')
* (60905015000, 'total_equino')
* (60609034000, 'total_ganosb')
* (60609034000, 'total_areatareas')
* (60609034000, 'total_riegotareas')
* (60609034000, 'total_bovino')
* (60609034000, 'total_equino')
* (60714034, 'total_bovino')
* (60714034, 'total_equino')



# Sobre la compatibilidad de los Códigos de Comunidad

En teoría:

- Códigos INE: DDMMAACCC
- Codigos planificación: DDMMAACCCXXX. Los XXX serán siempre '000' en las comunidades y solo varían en las ciudades principales refiréndose a los barrios. 
- Códigos de comunidades en Fonsagua: DDMMAACC
- Códigos de caserios en Fonsagua: DDMMAACC

El que los códigos de fonsagua tengan sólo 8 caracteres fue un error en su momento y se debería volver a usar 9 caracteres. Esto no es problemático y no hay ninguna dificuldad ni incoherencia con transformar los códigos de `caserios` y `comunidades` de Fonsagua a 9 caracteres mediante la función `nasmar.adjust_code_to_9_from_8`. Durante todo el proceso se usarán 9 caracteres para Fonsagua y cuando se vuelquen a la base de datos se volverá a 8 para no romper la aplicación.

Pero con los códigos de planificación hay algún problema sobre la teoría:

* No todos los códigos tienen 12 caracteres sino que tenemos algunos de 9
* No todos los códigos coinciden bien con los códigos de caserios o comunidades que tenemos en la base de datos (esto se analiza más adelante)
* El campo `nombre_comunidad` de los PDM tiene como primeros caracteres parte del código de comunidad (la parte referida a la comunidad o barrio) pero esto no siempre se cumple (se analiza más adelante)

In [3]:
# Conteo de cuantas longitudes de cógidos distintas tenemos
# merged.index.to_series().str.len().unique()
conteo_por_longitud_del_codigo = merged.index.to_series().str.len().value_counts()
display(conteo_por_longitud_del_codigo)

# Ajustamos las que tienen códigos de 8 y de 11 dígitos
# Esto es porque en el csv se está usando un entero en lugar de un texto para el
# código de comunidad y desprecia los '0' a la izquierda. Para 'Choluteca' se está
# usando '6' en lugar de '06' así que lo ajustamos
# El startswith '6' es un _double check_ para realmente comprobar que no hay errores

merged.reset_index(inplace=True)

# Actualizamos a 12 caracteres las que tienen 11
condicion_len_11 = (merged["codigo_comunidad"].str.len() == 11) & (merged["codigo_comunidad"].str.startswith('6'))
merged.loc[condicion_len_11, "codigo_comunidad"] = merged.loc[condicion_len_11, "codigo_comunidad"].str.zfill(12)

# Actualizamos a 9 caracteres las que tienen 8
condicion_len_8 = (merged["codigo_comunidad"].str.len() == 8) & (merged["codigo_comunidad"].str.startswith('6'))
merged.loc[condicion_len_8, "codigo_comunidad"] = merged.loc[condicion_len_8, "codigo_comunidad"].str.zfill(9)

merged.set_index("codigo_comunidad", inplace=True)

# Mostramos de nuevo el conteo para asegurarnos de que todo es correcto
# y lanzamos un error si no es así
conteo_por_longitud_del_codigo = merged.index.to_series().str.len().value_counts()
display(conteo_por_longitud_del_codigo)
if (len(conteo_por_longitud_del_codigo) != 2) or (12 not in conteo_por_longitud_del_codigo.index) or (9 not in conteo_por_longitud_del_codigo.index):
    raise Exception("El conteo de longitudes de código no es correcto")



# Y reordenamos los datos en función del código
merged.sort_index(ascending=True, axis='index', inplace=True)


12    296
11    270
8      27
9       9
Name: codigo_comunidad, dtype: int64

12    566
9      36
Name: codigo_comunidad, dtype: int64

# Sobre el campo "Nombre de comunidad"

### Formato del campo

También hacemos algunas comprobaciones sobre el "Nombre de comunidad" para entender mejor los datos y detectar incoherencias.

Entre las cosas a comprobar está el formato del "Nombre de Comunidad", ie: 

```
YYY - EL NOMBRE REAL
011 - SAN BERNARDO CENTRO
```
    
Se comprueba primero que todos siguen el formato `YYY` un espacio en blanco (` `) un `-` y otro espacio en blanco (` `). Se ve que esto no se cumple en dos casos y se corrige.

En el análisis vemos que hay dos nombres que no siguen exactamente el formato porqué falta un espacio y se arregla.

### El código que aparece en el campo

También comprobamos si el (sub)código que forma parte del campo nombre es válido.

Se debe tener en cuenta que los "Códigos de Comunidad" en los PDM se construyen como `DDMMAACCCXXX`, donde los `XXX` serán `000` en las comunidades y solo varían en las ciudades principales refiréndose a los barrios. Por tanto el subcódigo que hemos denominado con las letras `YYY`, en el nombre de comunidad será el `CCC`, cuando el código sea `DDMMAACCC000` y será el `XXX` cuando `XXX != 000`

En el análisis observamos que hay 25 casos para los que esto no se cumple sin encontrar ningún patrón claro.

### Reformateo

Para tener una columna más útil a la hora de volcar los datos de nuevo se crea una columna:

* `meta_nombre_comunidad_mal`. Con un valor `True` para aquellas filas que no siguen el formato
* `nombre_comunidad_sin_codigo`. El nombre sin la parte del código

In [4]:
# Mostramos aquellos elementos donde no se sigue el formato "XXX - NOMBRE"
condicion = ~merged["nombre_comunidad"].str.contains('^\d{3} - .*', regex=True)
no_sigue_el_formato = merged.loc[condicion, ["nombre_comunidad", "nombre_municipio"]]
display(HTML('<h2 style="color:red;">Sin el formato "YYY - NOMBRE"</h2>'))
display(no_sigue_el_formato)

# Los cambiamos a mano
merged["nombre_comunidad"].mask(condicion, lambda x: x.str[:3] + " - " + x.str[5:], inplace=True)
# merged.at['060902011000', 'nombre_comunidad'] = '011 - SAN BERNARDO CENTRO'
# merged.at['060902041000', 'nombre_comunidad'] = '041 - MONTE CRISTO'

# Nos aseguramos de que es correcto
no_todos_siguen_el_formato = ~merged["nombre_comunidad"].str.contains('^\d{3} - .*', regex=True).any()
if no_todos_siguen_el_formato:
    raise Exception("No todos siguen el formato: 'YYY - NOMBRE'")

,nombre_comunidad,nombre_municipio
codigo_comunidad,,
060902011000,011 -SAN BERNARDO CENTRO,NAMASIGUE
060902041000,041 -MONTE CRISTO,NAMASIGUE


In [5]:
# Mostramos aquellos códigos y nombres para los que no se cumple que:
# * cuando acaba en '000' los caracteres 7 a 9 del código son iguales a los tres primeros del nombre
# * Cuando no acaba en '000' los tres últimos caracteres del código son iguales a los tres primeros del nombre

cond = (
    (merged.index.str[-3:] == '000') & (merged.index.str[-6:-3] != merged["nombre_comunidad"].str[:3])
    |
    (merged.index.str[-3:] != '000') & (merged.index.str[-3:] != merged["nombre_comunidad"].str[:3])
)
merged["meta_nombre_comunidad_mal"] = cond
meta_nombre_comunidad_mal = merged.loc[merged["meta_nombre_comunidad_mal"], ["nombre_comunidad", "nombre_municipio"]]
merged.loc[merged["meta_nombre_comunidad_mal"], ["nombre_comunidad", "nombre_municipio"]].to_excel("meta_nombre_comunidad_mal.xlsx")

display(HTML('<h3 style="color:red;">El subcódigo del "nombre de comunidad" no corresponde con parte del "código de comunidad"</h3>'))
display(meta_nombre_comunidad_mal)


# Creamos una nueva columna sin el subcódigo de comunidad
merged["nombre_comunidad_sin_codigo"] = merged["nombre_comunidad"].str[6:]

,nombre_comunidad,nombre_municipio
codigo_comunidad,,
060703001005,002 - Bº Independiente- Buena Vista,MARCOVIA
060703001007,003 - El Indio- Buena Vista,MARCOVIA
060703001009,001 - COL. BUENA VISTA,MARCOVIA
060714003,005 - BO. VALLE- MONJARAS,MARCOVIA
060714004,001 - BO. CEDEÑO- MONJARAS,MARCOVIA
060714005,002 - BO. FONSECA- MONJARAS,MARCOVIA
060714006,004 - BO. SAN JOSE- MONJARAS,MARCOVIA
060714008,006 - BO. SINAI- MONJARAS,MARCOVIA
060714010,003 - BO. INDEPENDIENTE- MONJARAS,MARCOVIA


In [6]:
# Y a continuación se muestra un resumen de los datos de entrada y estadísticas numéricas para todas las columnas

display(Markdown("# Mostramos un conteo de cuantos códigos tenemos por municipio"))
display(nasmar.number_of_elements_by_council(merged, "codigo_comunidad"))
# Y lanzamos un error en caso que haya códigos de municipios que no sean de NASMAR
nasmar.drop_elements_not_in_nasmar_councils(merged, "codigo_comunidad", raise_error = True, log_warn = True)


display(Markdown("# Mostramos un resumen de los datos unidos"))
display(merged)
# display(HTML(merged.to_html()))

display(Markdown("# Y de sus estadísticas numéricas"))
# display(merged.describe())
display(HTML(merged.describe().to_html()))  # muestra todos los datos


# display(merged.info())

# Mostramos un conteo de cuantos códigos tenemos por municipio

Counter({'0606': 129,
         '0607': 92,
         '0609': 76,
         '1701': 154,
         '1703': 27,
         '1706': 67,
         '1709': 57})

# Mostramos un resumen de los datos unidos

,nombre_comunidad,nombre_municipio,total_viviendas,total_familias,total_personasv,total_sectorpri,total_sectorsec,total_sectorter,total_sectorcom,total_sectorind,...,total_mujeres5a18,total_hombres5a18,total_mujeres18a60,total_hombres18a60,total_adultos,total_ancianosm60,total_ancianasm60,total_ancianos,meta_nombre_comunidad_mal,nombre_comunidad_sin_codigo
codigo_comunidad,,,,,,,,,,,,,,,,,,,,,
060601001001,001 - BO. EL CENTRO,EL TRIUNFO,17,22,69,0,0,0,0,0,...,10,13,16,21,39,2,1,3,False,BO. EL CENTRO
060601001002,002 - COL. BELLA HORIZONTE,EL TRIUNFO,71,77,301,4,0,0,2,0,...,60,43,67,88,165,10,8,18,False,COL. BELLA HORIZONTE
060601001003,003 - BO. EL TOLOLO,EL TRIUNFO,149,156,605,9,32,26,19,3,...,106,76,135,172,325,28,34,62,False,BO. EL TOLOLO
060601001004,004 - BO. BRISAS DEL RIO GUALE,EL TRIUNFO,71,88,289,17,2,1,4,1,...,48,45,57,87,154,11,19,30,False,BO. BRISAS DEL RIO GUALE
060601001005,005 - BO. EL CALVARIO,EL TRIUNFO,44,51,209,3,3,1,20,0,...,39,39,43,55,108,7,6,13,False,BO. EL CALVARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170910005000,005 - LAS LOMAS,SAN LORENZO,3,3,9,0,0,0,0,0,...,1,0,3,4,7,0,0,0,False,LAS LOMAS
170910006000,006 - LAS PILAS,SAN LORENZO,28,29,100,2,0,0,0,0,...,24,7,26,30,57,3,2,5,False,LAS PILAS
170910007000,007 - VOLCANCITO,SAN LORENZO,26,30,94,0,0,0,0,0,...,16,7,26,26,54,5,3,8,False,VOLCANCITO


# Y de sus estadísticas numéricas

,total_viviendas,total_familias,total_personasv,total_sectorpri,total_sectorsec,total_sectorter,total_sectorcom,total_sectorind,total_sectorser,total_ganosb,total_areatareas,total_tenenciapropia,total_tenenciaalquilada,total_riegotareas,total_bovino,total_aves,total_porcino,total_equino,total_tenenciahom,total_tenenciamuj,total_notenencia,total_matabobe,total_matbahareque,total_cocinalena,total_cocinaeco,total_cocinaelec,total_vivconele,total_vivconapub,total_vivcontelf,total_vivcontelm,total_vivconaguaa,total_vivconaguar,total_vivconaguall,total_vivconfosasi,total_vivconfosase,total_vivconlet,total_ninosm5,total_ninasm5,total_ninos,total_mujeres5a18,total_hombres5a18,total_mujeres18a60,total_hombres18a60,total_adultos,total_ancianosm60,total_ancianasm60,total_ancianos
count,602.000000,602.000000,602.00000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,601.000000,601.000000,602.000000,602.000000,601.000000,600.000000,602.000000,601.000000,599.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000
mean,82.367110,96.995017,321.41196,15.594684,4.335548,6.373754,1.533223,3.694352,38.285714,113.612313,51.159734,13.551495,7.265781,1.602329,30.310000,344.813953,23.339434,3.273790,37.102990,19.880399,9.081395,31.719269,5.759136,65.013289,3.054817,8.825581,55.257475,42.727575,4.066445,33.666113,31.264120,3.598007,1.818937,19.737542,17.375415,32.275748,16.573090,15.973422,126.805648,47.910299,46.348837,79.390365,87.255814,174.797342,13.089701,14.754153,27.843854
std,91.594907,107.342106,347.96640,21.086191,8.960276,14.347187,4.636152,10.292863,57.364306,265.378085,157.651190,24.007726,12.437362,10.066939,51.413547,392.053821,36.238993,6.298205,40.037163,25.999212,14.234137,43.794930,9.323014,68.859449,11.212602,22.881765,75.867902,63.513926,12.242376,46.968533,65.463775,8.085722,4.925952,25.872673,29.618741,39.280472,18.971881,18.457559,138.333210,51.517878,51.742881,85.895181,97.475494,191.641700,14.356360,18.243300,32.100749
min,1.000000,1.000000,3.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,28.000000,31.000000,105.00000,2.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,97.000000,2.000000,0.000000,11.000000,4.250000,1.000000,4.250000,0.000000,21.000000,0.000000,0.000000,10.000000,6.000000,0.000000,6.000000,2.000000,0.000000,0.000000,4.000000,2.000000,6.000000,4.000000,4.000000,39.000000,15.000000,14.000000,26.000000,29.000000,58.000000,5.000000,4.000000,9.000000
50%,55.000000,64.000000,214.50000,7.000000,1.000000,1.000000,0.000000,0.000000,17.000000,20.000000,2.000000,6.000000,3.000000,0.000000,11.000000,228.500000,11.000000,1.000000,24.000000,11.000000,5.000000,15.000000,2.000000,44.000000,0.000000,2.000000,31.500000,21.000000,0.000000,18.000000,9.500000,1.000000,0.000000,12.000000,7.000000,19.000000,11.000000,10.000000,83.000000,32.000000,30.000000,53.500000,58.000000,118.500000,9.000000,9.000000,18.000000
75%,103.750000,118.750000,418.50000,22.000000,4.000000,7.000000,1.000000,2.000000,44.000000,94.000000,27.000000,16.000000,9.000000,0.000000,39.000000,423.500000,32.000000,4.000000,48.750000,24.750000,11.000000,39.750000,7.000000,82.750000,2.000000,7.000000,69.750000,50.000000,3.000000,44.000000,31.750000,3.000000,2.000000,25.000000,20.750000,46.750000,23.000000,22.000000,172.750000,63.000000,61.750000,100.000000,109.000000,219.750000,

In [7]:
# Para el análisis de como emparejar los códigos de PDM con los de Caserios y
# Comunidades, creamos una nueva columna que sean sólo los 9 primeros dígitos
# del código de PDM
merged["sort_code"] = merged.index.str[:9]

# Y volcamos los datos a un excel.
merged.to_excel(excel_datos_pdm_path, sheet_name="Datos_PDM", na_rep="", header=True, index=True)